In [1]:
import lightgbm as lgb
import os
import pandas as pd
import numpy as np
import random
import gc
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

np.random.seed(2019)
random.seed(2019)
pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 280)
pd.set_option('display.max_colwidth', 150)
data_path = '/data/workspace/kimi/tencent_ads/2020/dataset'

In [2]:
train_df =pd.read_pickle('test4.pkl').drop(['index'], axis=1)
train_df = train_df.sort_values(by='user_id')
train_df.replace("\\N",-1,inplace=True)
train_df=train_df.astype(float,inplace=True)
print(train_df)

          user_id  active_days  click_times_total  industry_0  industry_1  industry_2  industry_3  industry_4  industry_5  industry_6  industry_7  industry_8  industry_9  industry_10  industry_11  industry_12  industry_13  industry_14  industry_15  industry_top1_0  \
0       3000001.0          7.0               11.0    2.532888    1.878715    1.682341    5.494179   -4.078212    5.800343    2.827987    5.240605    6.578812   -4.025118    -2.619265     2.496614    -0.016113    -0.293491    -3.742475    -1.982195         6.615202   
1       3000002.0         37.0               65.0   -1.831123   -1.709645   -0.813786    1.221696    0.296469    1.149126   -3.300779    1.260832   -2.453826   -1.773209    -2.491741     0.341830    -4.217973    -0.916762     1.460926     3.130889        -4.283564   
2       3000003.0         23.0               36.0   -3.716056   -1.042728   -3.551677   -2.050015   -0.921868    1.912847   -3.276425    3.905490    2.156450    1.718182     0.558707     0.953603 

In [3]:
final_train_x_df = train_df.drop(['user_id'], axis=1)
#final_train_x_df = train_df.drop(['age','user_id','gender','active_days'], axis=1)
#X_train, X_test, y_train, y_test = train_test_split(final_train_x_df, final_train_y_df, test_size=0.20, random_state=42)
#train_data_show_df =train_df.drop(['active_days', 'click_times_total','max_clicked_industry_cnt','clicked_industry','clicked_advertiser','max_clicked_advertiser_cnt','max_clicked_industry_ratio','max_clicked_advertiser_ratio'], axis=1)
#print(X_train)
num_normal_features = ['_clicks_max_click_cnt','_max_clicked_ratio','_clicks_min_click_cnt','_min_clicked_ratio','_clicks_len','_clicks_mean','_clicks_median','_clicks_std']
num_date_features  = [ '_clicks_max_click_cnt', '_clicks_min_click_cnt','_clicks_len','_clicks_mean','_clicks_median','_clicks_std']
num_features = ['click_times_total'] +\
                [f'date{i}'  for i in num_date_features] + \
                [f'wday{i}'  for i in num_date_features] + \
                [f'month{i}'  for i in num_date_features] + \
                 [f'product_id{i}'  for i in num_normal_features] + \
                 [f'product_category{i}'  for i in num_normal_features] + \
                [f'industry{i}'  for i in num_normal_features] + \
                [f'advertiser_id{i}'  for i in num_normal_features]

#print(num_features)

c_features = ['industry_clicks_max_click','industry_clicks_min_click',
              'advertiser_id_clicks_max_click','advertiser_id_clicks_min_click',
              'product_id_clicks_max_click','product_id_clicks_min_click',
              'product_category_clicks_max_click','product_category_clicks_min_click',
             ]
features= num_features + c_features
features= [f"industry_{i}" for i in range(16)] + [f"advertiser_id_{i}" for i in range(64)] +['active_days','click_times_total'] + [f"industry_top1_{i}" for i in range(16)] + [f"advertiser_id_top1_{i}" for i in range(64)]
#train_data = lgb.Dataset(final_train_x_df, label=final_train_y_df, feature_name=[   'max_clicked_industry', 'max_clicked_advertiser_id' ], categorical_feature=['max_clicked_industry','max_clicked_advertiser_id'])
#train_data = lgb.Dataset(X_train, label=y_train, feature_name=features, categorical_feature=c_features,free_raw_data=False)
#train_data = lgb.Dataset(final_train_x_df, feature_name=features,free_raw_data=False)
#eval_data = lgb.Dataset(X_test, label=y_test, feature_name=features, categorical_feature=c_features,free_raw_data=False,reference=train_data)


In [4]:

gbm =  lgb.Booster(model_file='age_emb.txt')
print('Feature importances:', list(gbm.feature_importance()))

Feature importances: [5935, 4714, 5011, 6100, 4995, 4713, 4871, 4957, 4552, 5166, 5148, 5310, 5596, 5672, 4948, 4953, 1102, 809, 718, 939, 752, 918, 698, 657, 786, 745, 707, 834, 840, 921, 759, 754, 5588, 5893, 7574, 6359, 9363, 5602, 7434, 5473, 6786, 5666, 5836, 7177, 7565, 7084, 7593, 6241, 6838, 6587, 5424, 5777, 6825, 7694, 9209, 8536, 5408, 5769, 5054, 6677, 6206, 5563, 7040, 7572, 5135, 5715, 6829, 6124, 7332, 7231, 7414, 7896, 5730, 6332, 6730, 8383, 5389, 8185, 4647, 5171, 4948, 6954, 8109, 6178, 6681, 6368, 4596, 5146, 6627, 7037, 5331, 5951, 5996, 7356, 7551, 5207, 4763, 6765, 1479, 1632, 1674, 1625, 1703, 1433, 1563, 1607, 1483, 1726, 1893, 1589, 1622, 1508, 1591, 1603, 1431, 1781, 1785, 1693, 1727, 1681, 1527, 1563, 1414, 1605, 1423, 1728, 1624, 2190, 1477, 1726, 1530, 1686, 1531, 1781, 1586, 1547, 1679, 1584, 1451, 1730, 1556, 1357, 1653, 1613, 1530, 1820, 1771, 1602, 1633, 1646, 1301, 1568, 1741, 1636, 1553, 1656, 1690, 1787, 1699, 1434, 1525, 1868]


In [5]:
y_pred = gbm.predict(final_train_x_df)
print(y_pred)
for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0
print(y_pred)


[[5.07787569e-03 4.71176923e-02 4.34768533e-01 ... 4.13411532e-04
  1.00035453e-04 2.59898909e-05]
 [4.49019101e-03 5.02663659e-02 8.15104289e-02 ... 6.83579901e-02
  9.36846142e-03 3.47833104e-04]
 [1.01524855e-02 1.34200138e-01 2.49216690e-01 ... 3.89833336e-02
  1.13022930e-02 1.80122671e-03]
 ...
 [1.89290903e-01 6.97651011e-01 1.08981042e-01 ... 9.54126995e-05
  3.20272257e-05 4.12977410e-06]
 [1.48347357e-02 8.00560394e-01 1.68877636e-01 ... 7.25966603e-05
  3.19105323e-05 3.26152245e-06]
 [2.80851852e-03 2.29690014e-02 2.30213195e-01 ... 5.35699251e-03
  7.34487656e-04 1.05686156e-04]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
ret = []
for user_id,age in zip(range(1000000),y_pred):
    ret.append([int(user_id+3000001),int(age.tolist().index(1) + 1)])

ret_df = pd.DataFrame(ret,columns=['user_id','predicted_age'])
ret_df.to_pickle("age_emb_out.pkl")
print(ret_df)
print(ret_df['predicted_age'].value_counts())

        user_id  predicted_age
0       3000001              3
1       3000002              6
2       3000003              4
...         ...            ...
999997  3999998              2
999998  3999999              2
999999  4000000              4

[1000000 rows x 2 columns]
3     405658
4     210863
2     153494
       ...  
1      17189
9      12085
10      4968
Name: predicted_age, Length: 10, dtype: int64
